# Raw staking events to staker activity table Script
This script converts the staking events data returned by [Threshold Network Staking Event Dune query](./ThresholdNetworkStakingEvents.sql) to a table that contains stake activity history.


In [1]:
import pandas as pd

staking_events = pd.read_csv(
    'StakingEvents2024-12-18.csv', dtype={'amount': str})

staking_events["amount"] = staking_events["amount"].apply(int)

timestamp_list = staking_events['timestamp']
st_prov_list = staking_events.staking_provider.unique().tolist()

stakes_history = pd.DataFrame(index=staking_events.index, columns=['timestamp']+st_prov_list)

for index, row in staking_events.iterrows():
    if index == 0:
        stakes_history.iloc[index] = 0
        stakes_history.loc[index, 'timestamp'] = row.timestamp
        stakes_history.loc[index, row.staking_provider] = row.amount
    else:
        stakes_history.iloc[index] = stakes_history.iloc[index - 1]
        stakes_history.loc[index, 'timestamp'] = row.timestamp
        if row.event == "staked":
            stakes_history.loc[index, row.staking_provider] = row.amount
        elif row.event == "toppedUp":
            stakes_history.loc[index, row.staking_provider] += row.amount
        elif row.event == "unstaked":
            stakes_history.loc[index, row.staking_provider] -= row.amount

stakes_history.to_csv('StakesHistory2024-12-18.csv')